# (Legacy) Cargas de abrigados por template

O template loader é uma carga realizada com base no template `abrigados_template_v1.xlsx`.

> **Template file**
>
> abrigados\template_loader\templates\abrigados_template_v1.xlsx


In [ ]:
%load_ext blackcellmagic

## Instalação de dependências


In [1]:
## Importação de dependências
import re
import os
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import uuid
import sys
from datetime import datetime
from selector import environment_selector
from firestore_model_abrigado_legacy import AbrigadoLegacyEntity, MembroFamiliarLegacyEntity, DocumentoLegacyEntity, ResponsavelLegacyEntity
from firestore_upload_data import upload_to_firestore, upload_to_firestore_checking_duplicated
from firestore_query_batch import query_batch
from nome import normalize_nome
from datetime_xlsx import format_xlsx_datetime
from documento import format_documento
from additional_info import format_additional_info, sanitize_additional_info
from abrigo_dict import get_abrigo_info

# Configuração


### Environment Selector


In [ ]:
environment_selector()

#### Configura chaves e parâmetros
Carrega variáveis de ambiente e seta parâmetros como nome de collection.

In [ ]:
# Firebase Firestore Config
base_path = os.getenv("SOSRS_BASEPATH")
sosrs_environment = os.getenv("SOSRS_ENVIRONMENT")
sosrs_firestore_keyfile = os.getenv("SOSRS_FIRESTORE_KEYFILE")
sosrs_firestore_path = os.path.join(base_path, sosrs_firestore_keyfile)

# Firestore Collection
collection_name = "abrigado"

#### Autenticação do notebook
Inicializa comunicação com Firestore. É necessário selecionar o ambiente no Environment Selector e ter configurado no `.env` os caminhos para o arquivo do `service-account do GCP`.

In [ ]:
# Inicialização do Firebase
cred = credentials.Certificate(sosrs_firestore_path)
firebase_admin.initialize_app(cred)
clientFirestore = firestore.client()

# Template loader - Carregamento de abrigados baseado em template
Utiliza um template acordado no AbrigosRS para realizar a carga de dados de diferentes fontes.
Os fluxos tratados nesse notebook são:
- Importação do template legado import_abrigados_template_v1.xlsx com retrocompatibilidade com o modelo atual da collection no Firestore.
- Importação do template import_abrigados_template_v2.googlesheets.md com uma modelagem normalizada e baseada nos dados do templates, trazendo mais riqueza, qualidade e sentido para os dados.

### Nomes de colunas
Nomeia as colunas do template para facilitar a leitura e entendimento no notebook.

In [ ]:
# Define o nome das colunas do arquivo de entrada
column_names = [
    "timestamp",
    "abrigo",
    "data_entrada",
    "responsavel_nome",
    "responsavel_nome_mae",
    "responsavel_data_nascimento",
    "responsavel_rg",
    "responsavel_cpf",
    "responsavel_nis",
    "responsavel_titulo_eleitor",
    "responsavel_outros_documentos",
    "telefone",
    "endereco",
    "bairro",
    "cidade",
    "beneficio_informacoes",
    "pessoa1_nome",
    "pessoa1_parentesco",
    "pessoa1_data_nascimento",
    "pessoa2_nome",
    "pessoa2_parentesco",
    "pessoa2_data_nascimento",
    "pessoa3_nome",
    "pessoa3_parentesco",
    "pessoa3_data_nascimento",
    "pessoa4_nome",
    "pessoa4_parentesco",
    "pessoa4_data_nascimento",
    "pessoa5_nome",
    "pessoa5_parentesco",
    "pessoa5_data_nascimento",
    "pessoa6_nome",
    "pessoa6_parentesco",
    "pessoa6_data_nascimento",
    "pessoa7_nome",
    "pessoa7_parentesco",
    "pessoa7_data_nascimento",
    "pessoa8_nome",
    "pessoa8_parentesco",
    "pessoa8_data_nascimento",
    "pessoa9_nome",
    "pessoa9_parentesco",
    "pessoa9_data_nascimento",
    "pessoa10_nome",
    "pessoa10_parentesco",
    "pessoa10_data_nascimento",
    "redes_de_apoio_informacoes",
    "responsavel_preenchimento",
    "informacoes_adicionais",
]

# Define os campos do tipo data que devem ser tratados
datetime_fields = [
    "timestamp",
    "data_entrada",
    "responsavel_data_nascimento",
    "pessoa1_data_nascimento",
    "pessoa2_data_nascimento",
    "pessoa3_data_nascimento",
    "pessoa4_data_nascimento",
    "pessoa5_data_nascimento",
    "pessoa6_data_nascimento",
    "pessoa7_data_nascimento",
    "pessoa8_data_nascimento",
    "pessoa9_data_nascimento",
    "pessoa10_data_nascimento",
]

# Define campos manipulados no processamento
search_field_name = "search_field_name"
abrigo_nome = "abrigo"

### Configuração de datasources


In [ ]:
# Configuração do datasource
datasource_file = "./pipeline/.working/eeem_alcides_cunha_template_abrigados_abrigados.xlsx"
missing_abrigo = "./pipeline/.working/missing_abrigo.json"
master_data_sheet = "abrigados"
number_of_columns = 49

### Transformação de dados

##### Criação do dataclass


In [ ]:
# Cria uma entidade AbrigadoLegacyEntity a partir de uma linha do CSV


def create_abrigado_entity(csv_row):

    abrigo_nome, abrigo_id = get_abrigo_info(csv_row["abrigo"])

    # Se abrigo for vazio ou NaN, seta como 'abrigo nao informado na importacao'
    if csv_row["abrigo"] == "" or pd.isna(csv_row["abrigo"]):
        abrigo_nome, abrigo_id = get_abrigo_info("abrigo nao informado na importacao")

    abrigado_entity = AbrigadoLegacyEntity(
        nome=csv_row["responsavel_nome"],
        search_field_name=normalize_nome(csv_row["responsavel_nome"]),
        dataNascimento=csv_row["responsavel_data_nascimento"] if pd.notna(csv_row["responsavel_data_nascimento"]) else None,
        endereco=csv_row["endereco"] if pd.notna(csv_row["endereco"]) else None,
        temDocumento=None,
        acompanhadoMenor=None,
        temRenda=None,
        renda=None,
        temHabitacao=None,
        situacaoMoradia=None,
        necessidadesImediatas=None,
        cadastradoCadUnico=None,
        abrigoId=abrigo_id,
        abrigoNome=abrigo_nome,
        documentos=[format_documento(csv_row["responsavel_cpf"])],
        additional_info=[
            format_additional_info("data_entrada", csv_row["data_entrada"]),
            format_additional_info("telefone", csv_row["telefone"]),
            format_additional_info("bairro", csv_row["bairro"]),
            format_additional_info("cidade", csv_row["cidade"]),
            format_additional_info("beneficio_informacoes", csv_row["beneficio_informacoes"]),
            format_additional_info("redes_de_apoio_informacoes", csv_row["redes_de_apoio_informacoes"]),
            format_additional_info("responsavel_preenchimento", csv_row["responsavel_preenchimento"]),
            format_additional_info("informacoes_adicionais", csv_row["informacoes_adicionais"]),
        ],
    )

    # Remove strings vazias
    abrigado_entity.additional_info = sanitize_additional_info(abrigado_entity.additional_info)

    return abrigado_entity


def append_membro_familiar_entity(nome, data_nascimento, grau_parentesco, abrigado_entity):

    # Criar um dicionário com valores válidos apenas
    kwargs = {}
    if pd.notna(nome) and str(nome).strip() != "":
        kwargs["nome"] = str(nome).strip()
    if pd.notna(data_nascimento) and str(data_nascimento).strip() != "":
        kwargs["data_nascimento"] = str(data_nascimento).strip()
    if pd.notna(grau_parentesco) and str(grau_parentesco).strip() != "":
        kwargs["grau_parentesco"] = str(grau_parentesco).strip()

    # Verifica se há pelo menos um campo válido
    if kwargs:
        membro_familiar = MembroFamiliarLegacyEntity(**kwargs)

        # Adiciona membro familiar à lista de membros familiares se existir a lista
        if hasattr(abrigado_entity, "grupoFamiliar"):
            if isinstance(abrigado_entity.grupoFamiliar, list):
                abrigado_entity.grupoFamiliar.append(membro_familiar)
            else:
                abrigado_entity.grupoFamiliar = [membro_familiar]
        else:
            abrigado_entity.grupoFamiliar = [membro_familiar]
    else:
        print("Skipping...Nenhum dado válido para criar um membro familiar.")

    return abrigado_entity


def append_responsavel_entity(nome, data_nascimento, abrigado_entity):

    responsavel = ResponsavelLegacyEntity(
        nome=nome if pd.notna(nome) and nome != "" else None,
        data_nascimento=data_nascimento if pd.notna(data_nascimento) and data_nascimento != "" else None,
        additional_info=None,
    )

    abrigado_entity.responsavel = responsavel

    return abrigado_entity


def append_documento_entity(cpf, rg, nis, titulo_eleitor, outros_documentos, abrigado_entity):
    documento_entity = DocumentoLegacyEntity(
        cpf=cpf if pd.notna(cpf) and cpf != "" else None,
        rg=rg if pd.notna(rg) and rg != "" else None,
        nis=nis if pd.notna(nis) and nis != "" else None,
        titulo_eleitor=titulo_eleitor if pd.notna(titulo_eleitor) and titulo_eleitor != "" else None,
        outros_documentos=outros_documentos if pd.notna(outros_documentos) else None,
    )

    abrigado_entity.responsavel.documentos = documento_entity

    return abrigado_entity


def transform_csv_data(csv_row):

    # Cria entidade abrigado
    abrigado_entity = create_abrigado_entity(csv_row)

    # Adiciona membros familiares se existirem
    for i in range(1, 11):
        if i != 2:
            append_membro_familiar_entity(
                csv_row.get(f"pessoa{i}_nome"), csv_row.get(f"pessoa{i}_parentesco"), csv_row.get(f"pessoa{i}_data_nascimento"), abrigado_entity
            )

        else:
            append_membro_familiar_entity(
                csv_row.get(f"pessoa{i}_nome"), csv_row.get(f"pessoa{i}_data_nascimento"), csv_row.get(f"pessoa{i}_parentesco"), abrigado_entity
            )

    # Adiciona responsável
    append_responsavel_entity(csv_row["responsavel_nome"], csv_row["responsavel_data_nascimento"], abrigado_entity)

    # Adiciona documentos
    append_documento_entity(
        csv_row["responsavel_cpf"],
        csv_row["responsavel_rg"],
        csv_row["responsavel_nis"],
        csv_row["responsavel_titulo_eleitor"],
        csv_row["responsavel_outros_documentos"],
        abrigado_entity,
    )

    return abrigado_entity

##### Função de manipulação de dados duplicados


In [ ]:
def filter_duplicated_names(df_for_filtering):
    # Consulta nomes
    duplicity_check_result = query_batch(clientFirestore, collection_name, df_for_filtering, search_field_name, "in", 10)

    # Filtrar para manter apenas entradas com um ID não None
    registered_names = {name: id for name, id in duplicity_check_result.items() if id is not None}

    # Lista de nomes já registrados
    registered_names_list = list(registered_names.keys())

    # Filtrar o DataFrame para remover nomes já registrados
    df_filtered = df_for_filtering[~df_for_filtering[search_field_name].isin(registered_names_list)]

    return df_filtered

### Execução

#### Abrigos não mapeados
Para facilitar o processo de integração e não ficar impacto pela falta de normalização do nome dos abrigos, existe um mapeamento de abrigos entre o que estão nas planilhas e o que está cadastrado no sistema.
Durante a importação, quando um abrigo não é encontrado no mapeamento, então ele é excluído do dataframe de processamento e é persistido em "./datasource/recurrent/fasc/incoming/missing_abrigo.json".


##### Persiste nomes elegíveis


In [ ]:
# Carrega o arquivo xlsx agregador
if os.path.exists(datasource_file):
    xlsx = pd.ExcelFile(datasource_file)
else:
    print(f"Datasource file not found. {datasource_file}")
    sys.exit(1)

# Define memória para abrigos não encontrados
missing_abrigos = pd.DataFrame()

try:
    # Faz a leitura da aba master_data_sheet
    data_source = pd.read_excel(xlsx, sheet_name=master_data_sheet, header=0, names=column_names, usecols=range(number_of_columns))

    # Remove linhas onde a coluna 'nome' é NaN ou vazia
    data_source = data_source[data_source["responsavel_nome"].notna() & (data_source["responsavel_nome"] != "")]

    # Preenche campos vazios com strings vazias e formata campos de data
    data_source.fillna("", inplace=True)
    format_xlsx_datetime(data_source, datetime_fields)

    # Define as colunas para verificar duplicatas
    duplicate_columns = [
        "responsavel_nome",
        "responsavel_nome_mae",
        "responsavel_data_nascimento",
        "responsavel_rg",
        "responsavel_cpf",
        "responsavel_nis",
    ]

    # Identifica duplicatas, mantendo a primeira ocorrência
    duplicates = data_source[data_source.duplicated(subset=duplicate_columns, keep=False)]

    # Filtra para manter apenas as duplicatas excluídas (ignora a primeira ocorrência)
    duplicates_excluded = duplicates.drop_duplicates(subset=duplicate_columns, keep="first")

    # Remove linhas duplicadas baseado nas colunas especificadas no DataFrame original
    data_source = data_source.drop_duplicates(subset=duplicate_columns, keep="first")

    if not duplicates_excluded.empty:
        print("Registros duplicados que foram excluídos:")
        print(duplicates_excluded)

    # Verifica se 'abrigo' é vazia ou NaN e 'responsavel_nome' é válida; se sim, define 'abrigo' como "sem abrigo"
    data_source.loc[
        (data_source["abrigo"].isna() | (data_source["abrigo"] == ""))
        & data_source["responsavel_nome"].notna()
        & (data_source["responsavel_nome"] != ""),
        "abrigo",
    ] = "abrigononaoinformadonaimportacao"

    # Salva as colunas de data temporariamente
    datetime_columns_data = data_source[datetime_fields].copy()

    # Adiciona uma coluna para verificar a existência do abrigo usando get_abrigo_info
    data_source["abrigo_exists"] = data_source[datetime_columns_data].apply(lambda x: get_abrigo_info(x)[1] is not None)

    # Define colunas para comparação de duplicatas, mantendo todas as colunas originais, exceto a coluna de busca específica
    comparison_columns = [col for col in data_source.columns if col not in datetime_fields and col != "search_field_name"]

    # Identifica registros duplicados usando as colunas definidas para comparação
    data_source["duplicate"] = data_source.duplicated(subset=comparison_columns, keep=False)

    # Filtra para encontrar registros válidos: não duplicados e com abrigo existente
    valid_dataframe = data_source[(data_source["abrigo_exists"]) & (~data_source["duplicate"])]

    # Identifica registros com abrigos ausentes
    missing_abrigos = data_source[~data_source["abrigo_exists"]]

    if not missing_abrigos.empty:
        print(f"Missing abrigos found: {missing_abrigos["search_field_name"].unique()}")
        print("Removing missing abrigos from new records.")
        # Salvar os abrigos ausentes para arquivo JSON (substitua 'missing_abrigo' pela variável correta com o caminho do arquivo)
        missing_abrigos.to_json(missing_abrigo, orient="records", lines=True)

        if valid_dataframe.empty:
            print("All records have missing abrigos. Exiting...")
            sys.exit(1)

    # Atualiza data_source para conter apenas registros válidos
    data_source = valid_dataframe.copy()

    # Opcionalmente, pode-se remover as colunas 'abrigo_exists' e 'duplicate' se não forem mais necessárias
    data_source.drop(columns=["abrigo_exists", "duplicate"], inplace=True)

    # Cria coluna search_field_nome
    data_source["search_field_name"] = data_source.apply(
        lambda row: normalize_nome(row["search_field_name"]), axis=1
    )


except ValueError:
    print(f"Error reading sheets. Check the aggreate spreadsheet in the documentation. {master_data_sheet}")
    sys.exit(1)


# Transforma os dados, valida e faz o upload
try:
    upload_to_firestore_checking_duplicated(clientFirestore, collection_name, data_source, transform_csv_data, filter_duplicated_names)
    print("Upload finished.")
except Exception as e:
    print(f"Error uploading data to Firestore: {e}")
    sys.exit(1)

print("Files processing finished.")